In [2]:
import nipype.pipeline.engine as pe                        # the workflow and node wrappers
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.ants import N4BiasFieldCorrection
from nipype.interfaces.io import SelectFiles, DataSink

import os



ModuleNotFoundError: No module named 'nipype'

In [ ]:
#2: Specify variables
base_dir = "/home/masoomeh/MR_Preprocess/"
output_dir = os.path.join(os.path.dirname(base_dir), "out")
in_file = "/home"   
subject_id = ["Brats17_CBICA_AQA_1"]

#3: Specify nodes
anat_input = pe.Node(IdentityInterface(fields=in_file, mandatory_inputs=True),name="anat_input")
biascor = pe.Node(N4BiasFieldCorrection(input_image = in_file,
                                        dimension=3,
        				bspline_fitting_distance=300,
       					shrink_factor=3,
        				n_iterations=[50, 50, 30, 20],
        				convergence_threshold=1e-6),
					name = "bias_correction")

#4: Create the workflow object, specify workflow 
wf = pe.Workflow(name='preproc')
wf.base_dir = '.'


#5: Input and output stream

templates = {'anat': 'data/{subject_id}/struct.nii'}
selectfiles = pe.Node(SelectFiles(templates,
                                  base_directory=base_dir), name="selectfiles")


datasink = pe.Node(DataSink(parameterization=False), name="datasink")
datasink.inputs.base_directory = "/home/masoomeh/MR_Preprocess/"
datasink.inputs.container = output_dir
substitutions = [('_subject_id_', '')]
datasink.inputs.substitutions = substitutions


infosource = pe.Node(IdentityInterface(fields=["subject_id"]), name="infosource")

wf.connect([
           # Connect SelectFiles and DataSink to the workflow
           (infosource, selectfiles, [("subject_id", "subject_id")]),
           (selectfiles, anat_input, [("anat", "in_file")]),
           (anat_input, biascor, [("in_file", "input_image")]),
           (anat_input, datasink, [("in_file", "output_anat")]),
            # input to datasink
           (biascor,   datasink    , [("output_image", "output_bias")]),
           ])
#6. run worflow
wf.write_graph()

wf.run()